In [39]:
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from model import Iris2LayerClassifier
import torch

device = "cuda" if torch.cuda.is_available() else "cpu"
# device = "cpu"

iris = load_iris()
X, y = iris.data, iris.target

X_train, X_test, y_train, y_test = train_test_split(
    X, y, random_state=42, test_size=0.33
)
X_train, X_test, y_train, y_test = (
    torch.tensor(X_train, device=device, dtype=torch.float32),
    torch.tensor(X_test, device=device, dtype=torch.float32),
    torch.tensor(y_train, device=device, dtype=torch.long),
    torch.tensor(y_test, device=device, dtype=torch.long),
)

In [40]:
import torch.optim as optim
from sklearn.metrics import accuracy_score
from collections import namedtuple
from torch.utils.data import DataLoader
import torch.nn as nn


def train_model(train_loader: DataLoader):
    model = Iris2LayerClassifier().to(device=device)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=0.005)
    model.train()
    num_epochs = 200
    for i in range(num_epochs):
        for inputs, targets in train_loader:
            outputs = model(inputs)
            loss = criterion(outputs, targets)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
        # if i % 10 == 0:
        #     print(f"Loss = {loss:4f}")
    # model.eval()
    # with torch.inference_mode():
    #     y_pred = model(X_test)
    #     _, predicted_labels = torch.max(y_pred,1)
    #     accuracy = accuracy_score(y_test.cpu(), predicted_labels.cpu())
    #     print(f"Accuracy = {accuracy}")
    return model.state_dict()

In [41]:
from torch.utils.data import TensorDataset, DataLoader

NUM_OF_MODELS = 500

train_dataset = TensorDataset(X_train, y_train)
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)

real_models = []
for i in range(NUM_OF_MODELS):
    model_info = train_model(train_loader)
    real_models.append(model_info)
    if (i + 1) % 50 == 0:
        print("trained {} models".format(i + 1))

trained 50 models
trained 100 models
trained 150 models
trained 200 models
trained 250 models
trained 300 models
trained 350 models
trained 400 models
trained 450 models
trained 500 models


In [42]:
import pickle
edit_pickle = True

if edit_pickle == True:
    # Save the list to a file
    with open('2_layer_real_models.pickle', 'wb') as f:
        pickle.dump(real_models, f)
        print("Saved trained models")

Saved trained models


In [43]:
import random

r = random.randint(0, 499)
model_test = Iris2LayerClassifier().to(device)
model_test.load_state_dict(real_models[r])
model_test.eval()
with torch.inference_mode():
    y_pred = model_test(X_test)
    _, predicted_labels = torch.max(y_pred, 1)
    accuracy = accuracy_score(y_test.cpu(), predicted_labels.cpu())
    print(f"Accuracy = {accuracy}, model number = {r}")

Accuracy = 0.98, model number = 464
